# Carbon Market SDE Simulation
This notebook simulates price dynamics in carbon emission trading markets using a nonlinear stochastic differential equation (SDE) with jump diffusion and policy feedback.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

In [ ]:
# Simulation parameters
T = 1.0
N = 500
dt = T / N
time = np.linspace(0, T, N+1)
P0 = 1.0

# Model parameters
alpha = 0.5
delta = 2.0
g0 = 0.4
lam = 2.0
beta = 0.3
sigma0 = 0.2
gamma = 1.0
eta = 0.1
theta = 0.3
kappa = 0.5
lambda0 = 10


## Simulate a Single Price Path

In [ ]:
P = np.zeros(N+1)
P[0] = P0
jumps = []

np.random.seed(42)
for t in range(N):
    current_t = time[t]
    gt = g0 * np.exp(-lam * current_t)
    mu = alpha * (1 - P[t]**delta) + gt + beta * P[t] * gt
    dWt = np.random.normal(0, np.sqrt(dt))
    dNt = np.random.poisson(lambda0 * dt)
    sigma = sigma0 * (1 + gamma * np.sin(P[t])) + eta * P[t] * dNt
    J = theta * P[t]**(1 + kappa) * (1 + lam * gt) * dNt
    P[t+1] = P[t] + mu * dt + sigma * dWt + J
    if dNt > 0:
        jumps.append(t)

plt.figure(figsize=(10, 5))
plt.plot(time, P, label='Simulated Price Path')
plt.scatter(time[jumps], P[jumps], color='red', label='Jumps')
plt.title('Optimized Single Simulated Price Path')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Final Price Distribution from Multiple Paths

In [ ]:
num_paths = 1000
final_prices = []
np.random.seed(123)
for i in range(num_paths):
    P = np.zeros(N+1)
    P[0] = P0
    for t in range(N):
        current_t = time[t]
        gt = g0 * np.exp(-lam * current_t)
        mu = alpha * (1 - P[t]**delta) + gt + beta * P[t] * gt
        dWt = np.random.normal(0, np.sqrt(dt))
        dNt = np.random.poisson(lambda0 * dt)
        sigma = sigma0 * (1 + gamma * np.sin(P[t])) + eta * P[t] * dNt
        J = theta * P[t]**(1 + kappa) * (1 + lam * gt) * dNt
        P[t+1] = P[t] + mu * dt + sigma * dWt + J
    final_prices.append(P[-1])

plt.figure(figsize=(10, 5))
plt.hist(final_prices, bins=50, color='skyblue', edgecolor='black', density=True)
plt.title('Final Price Distribution from 1000 Simulated Paths')
plt.xlabel('Final Price')
plt.ylabel('Density')
plt.grid(True)
plt.tight_layout()
plt.show()